In [1]:
import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

import random
import skimage as sk
from skimage import transform

# mount google drive locally
from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/My Drive/AAIIS Project/data/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def loadData(paths):
  x = []
  y = []
  wt = []
  weight = []
  label = pd.read_csv("trainLabels.csv")
  for img in tqdm(os.listdir(paths)):
    path = os.path.join(paths, img)
    image = cv2.imread(path)#cv2.IMREAD_GRAYSCALE
    tmp = label[label["filename"].str.contains(img)]
    x1, x2, w, h = list(tmp["bounding_box"])[0][1:-1].split(',')
    image = image[int(x2):int(h), int(x1):int(w)]
    image = cv2.resize(image,(96, 96))
    image = image.reshape(3,96,96)
    x.append(image)
    y.append(tmp["type"])
    wt.append(tmp["type"].values)
   
  classes, counts = np.unique(wt, return_counts=True)
  for i in range(len(classes)):
    weight.append(len(y)/counts[i])
    
  return x, y, weight

In [3]:
x, y, weight = loadData('images/')

100%|██████████| 2100/2100 [00:28<00:00, 73.95it/s]


In [0]:
# model for classification
CarClassification = nn.Sequential(nn.Conv2d(3,100, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                                  nn.BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                                  nn.ReLU(inplace=True),
                                  nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                                  nn.Conv2d(100,100, kernel_size=(3, 3), stride=(1,1), padding=(1,1)),
                                  nn.ReLU(inplace=True),
                                  nn.BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                                  nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                                  nn.Conv2d(100,50, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                                  nn.BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                                  nn.ReLU(inplace=True),
                                  nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                                  nn.Conv2d(50,100,kernel_size=(12,12), stride=(1,1), padding=0),
                                  nn.ReLU(inplace=True),
                                  nn.Conv2d(100,5,kernel_size=(1,1), stride=(1,1), padding=0)
                                 )

In [0]:
class CarBrand:
  def __init__(self, x, y, transform):
    self.x = x
    self.y = y
    self.transform = transform
  
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, idx):
    image = self.x[idx]
    label = self.y[idx]
    label = int(label.map({'audi':0, 'bmw':1, 'chevrolet':2, 'dodge':3, 'ford':4, 'hyundai':5 }))
    data = {'x':image, 'y':label}
    if self.transform:
        data = self.transform(data)
    return data
  
class CarType:
  def __init__(self, x, y, transform):
    self.x = x
    self.y = y
    self.transform = transform
  
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, idx):
    image = self.x[idx]
    label = self.y[idx]
    label = int(label.map({'convertible':0, 'coupe':1, 'hatchback':2, 'sedan':3, 'suv':4 }))
    data = {'x':image, 'y':label}
    if self.transform:
        data = self.transform(data)
    return data
  
class ToTensor(object):
  def __call__(self, data):
    x, y = torch.from_numpy(data['x']), torch.tensor(data['y'])
    return {'x': x.type(torch.DoubleTensor),
                'y': y}
  
class Normalize(object):
  def __call__(self, data):
    return {'x': data['x']/255, 'y': data['y']}
  
class Augmentation(object):
  def __call__(self, data):
    x, y = data['x'], data['y']
    i = random.randint(2,3)
    #i=1
    if i == 1:
      return {'x':x, 'y':y}
    if i ==3:
      random_degree = random.uniform(-25, 25)
      return {'x':sk.transform.rotate(x, random_degree), 'y':y}
    if i == 2:
      return {'x':sk.util.random_noise(x), 'y':y}
    if i == 4:
      return {'x':x[:, ::-1], 'y': y}

In [0]:
# convert model parameters to work with gpu
CarClassification.cuda()
loss_func = nn.CrossEntropyLoss(weight = torch.tensor(weight)).cuda()
#loss_func = nn.CrossEntropyLoss().cuda()
#Adam optimizer
optimizer = optim.Adam(CarClassification.parameters(), lr=0.001)

In [7]:
weight

[6.158357771260997,
 5.965909090909091,
 17.94871794871795,
 2.6717557251908395,
 4.166666666666667]

In [0]:
def error_rate(y, t):
  return np.mean(y != t)

def train(trainLoader, valLoader, x_,y_, iter):
  for i in range(iter):
    train_loss = 0.0
    val_loss = 0.0
    train_error = 0.0
    val_error = 0.0
    print("iteration : {}".format(i))
    for j, data in enumerate(trainLoader, 0):
      x, t = data['x'], data['y']
      
      # convert tensor to work with gpu
      x = x.type(torch.cuda.FloatTensor)
      t = t.type(torch.cuda.LongTensor)
      
      optimizer.zero_grad()
      y = CarClassification(x)
      y = y.reshape(-1,5)
      loss = loss_func(y, t)
      
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
      
      #train_error += error_rate(torch.argmax(y), t)
      
    for k, data in enumerate(valLoader, 0):
      x, t = data['x'], data['y']
      # convert tensor to work with gpu
      x = x.type(torch.cuda.FloatTensor)
      t = t.type(torch.cuda.LongTensor)
      
      y = CarClassification(x)
      y = y.reshape(-1,5)
      loss = loss_func(y, t)
      val_loss += loss.item()
      
      #val_error += error_rate(torch.argmax(y), t)
      
    print("Train Loss:{} || Validation Loss:{}".format(train_loss/2100, val_loss/169))
    if i>=15:
      dataa = CarType(x_, y_, transform= transforms.Compose([Normalize(),Augmentation(), ToTensor()]))
      trainLoader = DataLoader(dataa, batch_size=300, shuffle=True, num_workers=4)

  print("done")

In [0]:
data = CarType(x, y, transform= transforms.Compose([Normalize(), ToTensor()]))
train_dataloader = DataLoader(data, batch_size=300, shuffle=True, num_workers=4)

x_test, y_test, dummy = loadData('test/')
data_test = CarType(x_test, y_test, transform= transforms.Compose([Normalize(), ToTensor()]))
val_dataloader = DataLoader(data_test, batch_size=300, shuffle=False, num_workers=4)

 56%|█████▌    | 95/169 [00:01<00:00, 75.94it/s]

In [0]:
train(train_dataloader, val_dataloader,x,y, 60)

In [0]:
y_ = []
for i in range(169):
  x_ = data_test[i]['x'].reshape(-1,3,96,96).type(torch.cuda.FloatTensor)
  y_.append(CarClassification(x_))

In [0]:
ynew = []
for i in range(169):
  ynew.append(torch.argmax(y_[i]))

In [0]:
_y = []
for i in range(169):
  _y.append(data_test[i]['y'].type(torch.cuda.LongTensor))

In [0]:
#for i in range(169):
#  print("{} :::: {}".format(_y[i], ynew[i]))
error_rate(np.array(_y), np.array(ynew))